---
title: Notes on NeoBERT
#subtitle: "..."
description: "My study notes on the 'NeoBERT: A Next-Generation BERT' paper and explorations with the model"
date: 2025-06-25
toc: true
---

The paper "[NeoBERT: A Next-Generation BERT](https://arxiv.org/abs/2502.19587)" (2025) by Lola Le Breton, Quentin Fournier, Mariam El Mezouar, John X. Morris, Sarath Chandar introduces a new encoder model with updated architecture, training data, and pre-training methods and is intended as a strong backbone model.

> [W]e introduce NeoBERT, a next-generation encoder that redefines the capabilities of bidirectional models by integrating state-of-the-art advancements in architecture, modern data, and optimized pre-training methodologies.

These are the most important highlights at a first glance:

- **Performance:** It performs state-of-the-art on MTEB for its parameter class against other backbone models like BERT, NomicBERT, or ModernBERT base.
- **Size:** The model is medium-sized with 250M parameters
- **Sequence length:** Compared to BERT and RoBERTa (512) it has an extended context length of 4,096 tokens similar to NomicBERT (2,048) and ModernBERT (8,192)
- **Speed:** NeoBERT is faster than ModernBERT in inference speed, despite being 100M parameters larger than the ModernBERT base
- **Dimensions:** NeoBERT maintains same hidden size as base models (768) allowing for seamless plug-and-play

And here are the relevant links:

- arXiv: [https://arxiv.org/abs/2502.19587](https://arxiv.org/abs/2502.19587)
- Model on Hugging Face: [https://huggingface.co/chandar-lab/NeoBERT](https://huggingface.co/chandar-lab/NeoBERT)
- Code: [https://github.com/chandar-lab/NeoBERT](https://github.com/chandar-lab/NeoBERT)


## Motivation
The motivation behind is paper is the fact that encoders have not received as much love as LLMs in recent years, although they are equally important for downstream applications, like RAG systems.

Today's LLMs are capable of in-context learning and reasoning because of advancements in architecture, training data, pre-training, and fine-tuning.
While there has been research on fine-tuning methods for pre-trained encoders (e.g., [GTE](Li et al., 2023b) or [jina-embeddings](Sturua et al., 2024)), they are applied to older base models, like [BERT](TODO link) from 2019.
Therefore, the authors see a lack of updated open-source
base models to apply these new fine-tuning techniques to.

> As a result, there is a dire need for a new generation of BERT-like pre-trained models that incorporate up-to-date knowledge and leverage both architectural and training innovations, forming stronger backbones for these more advanced fine-tuning procedures.

Recent work on modernizing these base models are NomicBERT and ModernBERT, which this paper takes inspiration from:

- NomicBERT:
    - Paper: [https://arxiv.org/abs/2402.01613](https://arxiv.org/abs/2402.01613) (`nomic-bert-2048`, not to be confused with `nomic-embed-text-v1`)
    - Model: [https://huggingface.co/nomic-ai/nomic-bert-2048](https://huggingface.co/nomic-ai/nomic-bert-2048)
    - License: apache-2.0
    - Released: February 2025
- ModernBERT:
    - Paper: [https://arxiv.org/abs/2412.13663](https://arxiv.org/abs/2412.13663)
    - Model: [https://huggingface.co/answerdotai/ModernBERT-base](https://huggingface.co/answerdotai/ModernBERT-base)
    - Model: [https://huggingface.co/answerdotai/ModernBERT-large](https://huggingface.co/answerdotai/ModernBERT-large)
    - License: apache-2.0
    - Released: December 2024

## Key insights
The paper covers a lot of interesting nitty-gritty details on recent advancements around architecture choice, training data selection, and pre-training methods.
But if you step back, I think the key insights are that they are confirming what we already know from LLMs:

1. Training on a lot of good data = Better models
2. Increasing model size = Better models (even at small scale)

### Training on a lot of good data = Better models

According to the paper, the modification with the biggest improvement was changing the training data:

> [...] replacing Wikitext and BookCorpus with the significantly larger and more diverse RefinedWeb dataset **improved the score by +3.6%** [...]

They trained NeoBERT on [RefinedWeb](https://huggingface.co/datasets/tiiuae/falcon-refinedweb), which is a 2.8 TB large dataset. It contains 600B tokens and is 18 times larger than RoBERTa's training dataset.

> Following the same trend, we pre-trained NeoBERT on RefinedWeb (Penedo et al., 2023), a massive dataset containing 600B tokens, nearly 18 times larger than RoBERTa’s.

I think it's interesting that apparently the newer NomicBERT was trained on the same dataset as BERT with 13GB, while RoBERTa was trained on an extended dataset of 160 GB, and it's just now that this has been done to encoders, while this has been done to generative models for a while already.

> Recent generative models like the LLaMA family (Touvron et al., 2023; Dubey et al., 2024) have demonstrated that language models benefit from being trained on significantly more tokens than was previously standard. Recently, LLaMA-3.2 1B was successfully trained on up to 9T tokens without showing signs of saturation. Moreover, encoders are less sample-efficient than decoders since they only make predictions for masked tokens. **Therefore, it is reasonable to believe that encoders of similar sizes can be trained on an equal or even greater number of tokens without saturating.**

### Increasing model size = Better models (even at small scale)

The second most impactful modification was increasing the model size and finding an optimal depth-to-width ratio for the Transformer architecture:

> [...] while increasing the model size from 120M to 250M in M7 **led to a +2.9% relative improvement**.

So, NomicBERT and ModernBERT base both have around 150M parameters and are considered small-sized.
NeoBERT with 250M parameters can be considered medium-sized, so it makes sense that it performs better than smaller models.

But what's interesting is that they took the depth-to-width ratio into consideration when increasing the model size:

> In contrast, small language models like BERT, RoBERTa, and NomicBERT are instead in a width-inefficiency regime. To maximize NeoBERT’s parameter efficiency while ensuring it remains a seamless plug-and-play replacement, we retain the original BERT base width of 768 and instead increase its depth to achieve this optimal ratio.

So, they first increased the number of parameters to 250M with a depth-to-width ratio of 16 x 1056 (too wide) and then they optimized the depth-to-width ratio to 28 x 768 (more width-efficient).

> Note that to assess the impact of the depth-to-width ratio, we first scale the number of parameters in M7 to 250M while maintaining a similar ratio to BERT base, resulting in 16 layers of dimension 1056. In M8, the ratio is then adjusted to 28 layers of dimension 768.

This is nice, because by keeping the hidden size at 768, NeoBERT can be easily switched out for other base models:
> NeoBERT is designed for seamless adoption: it serves as a plug-and-play replacement for existing base models

What I also think is remarkable is that it has a much faster inference speed compared to both ModernBERT models despite it's size.
There's a nice figure in the paper showing the thoughput at different sequence length.
The figure is missing NomicBERT though, which I don't know why.

>  For extended sequences, NeoBERT significantly outperforms ModernBERT base, despite having 100M more parameters, **achieving a 46.7% speedup on sequences of 4, 096 tokens**.

## Old Encoders vs. Modern Encoders

The paper features a nice overview table of different characteristics between older encoders, like BERT (2019), RoBERTa (2019), and newer encoders, like ModernBERT (2024) and NomicBERT (2025), which shows their differences. Here I'm summarizing key differences between older and newer encoders that stood out to me:

| Configuration | Older Encoders  | Newer Encoders  |
| -- | --  | -- |
| Position encoding and sequence lengths | Absolute positional embeddings with a sequence length of 512 | RoPE for handling longer sequences of 2,048 to 8,192 |
| Masking rate | 15 % | Optimal masking rate was found to be between 20 to 40 % [by Wettig et al.](https://arxiv.org/abs/2202.08005) |
| Optimizer| Adam | AdamW |
| Training  | DDP | FlashAttention and other |
| Normalization  | Post-Layer Normalization | Pre-Layer Normalization (normalization layer is moved inside the residual connection of each feed-forward and attention block)|

## NeoBERT Example

Since NeoBERT is a state-of-the-art encoder, it can be used as an embedding model.
Let's take it for a spin, shall we?

I'm going to quickyl test how to create an embedding vector and then use it with the [Weaviate](https://github.com/weaviate/weaviate) vector database.

If you want to test it out yourself, you can open this blog as a Jupyter notebook in Colab.
<a target="_blank" href="https://colab.research.google.com/github/iamleonie/website/blob/main/blog/neobert.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Let's first install the required packages.
(I'm fixing the `transformers` version that has worked for me here.)

In [1]:
#| echo: true
#| output: false
!pip install transformers==4.41.0 torch xformers==0.0.28.post3
!pip install -U weaviate-client

  Using cached weaviate_client-4.15.4-py3-none-any.whl.metadata (3.7 kB)
  Using cached validators-0.34.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached authlib-1.6.0-py2.py3-none-any.whl.metadata (4.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.0/433.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.73.0
    Uninstalling grpcio-1.73.0:
      Successfully uninstalled grpcio-1.73.0


### Generate a single embedding

Then, let's try the sample code from the [Hugging Face model card](https://huggingface.co/chandar-lab/NeoBERT).

In [2]:
#| echo: true
#| output: false
from transformers import AutoModel, AutoTokenizer

model_name = "chandar-lab/NeoBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

# Tokenize input text
text = "NeoBERT is the most efficient model of its kind!"
inputs = tokenizer(text, return_tensors="pt")

# Generate embeddings
outputs = model(**inputs)
embedding = outputs.last_hidden_state[:, 0, :]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

model.safetensors:   0%|          | 0.00/981M [00:00<?, ?B/s]

Let's take a look at the generated output:

Nice. It looks like a vector embedding.

In [3]:
embedding[0].tolist()[:5]

[-1.9510374069213867,
 -0.9967884421348572,
 -1.2037752866744995,
 0.24297747015953064,
 -0.1231154128909111]

As you can see, NeoBERT creates vectors of dimension 768.

In [4]:
embedding.shape

torch.Size([1, 768])

### Use it for vector search

And last, but not least, let's use NeoBERT for vector search.
For this, we will spin up an embedded instance of Weaviate.

In [5]:
import weaviate, os

# Connect to an embedded Weaviate instance
client = weaviate.connect_to_embedded()

client.is_ready()

INFO:weaviate-client:Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.30.5/weaviate-v1.30.5-Linux-amd64.tar.gz
INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 2836


True

To use NeoBERT with Weaviate, we're going to use the 'Bring your own vectors' approach and therefore won't specify a vectorizer during the creation of the collection. We will only specify one property field `"Text"` here.

In [6]:
import weaviate.classes.config as wc

# Delete the collection if it already exists
if (client.collections.exists("TestCollection")):
    client.collections.delete("TestCollection")

collection = client.collections.create(
    name="TestCollection",
    vectorizer_config=wc.Configure.Vectorizer.none(),
    properties=[
        wc.Property(name="Text", data_type=wc.DataType.TEXT),
    ]
)

Since we are not using an automatic vectorizer, we have to write a small function to generate embeddings manually.

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

def get_embedding(source_text: str):
  inputs = tokenizer(source_text, return_tensors="pt")

  # Generate embeddings
  outputs = model(**inputs)
  embedding = outputs.last_hidden_state[:, 0, :]
  return embedding[0].tolist()

Since we're just playing around with NeoBERT, we will load a small toy dataset of Jeopardy questions into the database.

In [8]:
import weaviate.classes as wvc
import requests, json

url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

objs = []
for i, d in enumerate(data):
    objs.append(wvc.data.DataObject(
        properties={
            "Text": d["Question"],
        },
        vector=get_embedding(d["Question"])
    ))

collection.data.insert_many(objs)

BatchObjectReturn(_all_responses=[UUID('a3b03b73-b3d7-43c0-898d-cf89ec378e38'), UUID('7448f996-495e-4b2c-807a-8379c713c8cd'), UUID('b7b69200-4846-49f6-ac58-4da5f7618080'), UUID('33a409ec-812e-4a2c-8c94-56674f387ef8'), UUID('7c651044-7908-4de9-9197-d6a4bc914764'), UUID('3f2d02dd-36f1-44bc-9570-9ae2c76681de'), UUID('dcfb335b-4b6f-48ea-a6a1-4e9a18bc1584'), UUID('f4bbc154-3a76-4bf4-8f0d-6687e93113fb'), UUID('03763f4c-eabe-43a1-b882-8b56886b6a2a'), UUID('dba1534a-7783-4534-a4e9-2cc27cbb6f31')], elapsed_seconds=0.006937980651855469, errors={}, uuids={0: UUID('a3b03b73-b3d7-43c0-898d-cf89ec378e38'), 1: UUID('7448f996-495e-4b2c-807a-8379c713c8cd'), 2: UUID('b7b69200-4846-49f6-ac58-4da5f7618080'), 3: UUID('33a409ec-812e-4a2c-8c94-56674f387ef8'), 4: UUID('7c651044-7908-4de9-9197-d6a4bc914764'), 5: UUID('3f2d02dd-36f1-44bc-9570-9ae2c76681de'), 6: UUID('dcfb335b-4b6f-48ea-a6a1-4e9a18bc1584'), 7: UUID('f4bbc154-3a76-4bf4-8f0d-6687e93113fb'), 8: UUID('03763f4c-eabe-43a1-b882-8b56886b6a2a'), 9: UUID(

Now, we're all set and we can run a simple search query with the `near_vector` method.

In [11]:
response = collection.query.near_vector(
    near_vector=get_embedding("Animal living in Africa"),
    limit=3,
    include_vector = True
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties)
    print("Vector:", item.vector, "\n")

ID: b7b69200-4846-49f6-ac58-4da5f7618080
Data: {'text': 'The gavial looks very much like a crocodile except for this bodily feature'}
Vector: {'default': [-1.866025447845459, -1.3518006801605225, -1.514726161956787, 0.4027024805545807, 0.17218011617660522, 1.3396141529083252, -1.2356294393539429, -2.034353733062744, -0.27608299255371094, -1.2130968570709229, 1.6713166236877441, -1.2161169052124023, 0.9616807103157043, 1.0904563665390015, 1.1187479496002197, 1.3453131914138794, 0.06568765640258789, -1.528752326965332, 2.649778127670288, 1.0101542472839355, -0.42384618520736694, 0.08110082894563675, 0.9350497722625732, -1.3716142177581787, -2.0792810916900635, 1.9355051517486572, -0.43601953983306885, -1.5071309804916382, -1.229486346244812, -1.2989834547042847, 2.5945374965667725, 1.1732534170150757, 1.9968559741973877, -0.7341495752334595, -1.0044344663619995, -3.9257802963256836, 2.994337558746338, 1.2184511423110962, -1.0550923347473145, -2.4088385105133057, -1.2593684196472168, 3.80